In [77]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations as comb
from pprint import pprint
import pandas as pd


def graph_and_stats(fname):
    G = nx.Graph()
    self_loop_count = 0
    pos_count = 0
    neg_count = 0
    with open(fname, "r") as f:
        for line in f:
            reviewer, reviewee, weight = tuple(map(int, line.split(",")))
            self_loop_count += 1 if reviewer == reviewee else 0
            pos_count += 1 if weight == 1 else 0
            neg_count += 1 if weight == -1 else 0
            G.add_edge(reviewer, reviewee, weight=weight)
    return self_loop_count, pos_count, neg_count, G

self_loop_count, pos_count, neg_count, G = graph_and_stats("epinions.csv")

In [ ]:
weights = nx.get_edge_attributes(G, 'weight')

In [ ]:
triads = [x for x in nx.enumerate_all_cliques(G) if len(x) == 3]

In [ ]:
triads_and_weights = list(map(lambda x: list(map(lambda x: (x, weights[x]), comb(x, 2))), triads))

In [ ]:
from pprint import pprint
pprint(triads_and_weights[:20]) #ABBREVIATED

In [ ]:
print("NUMBER OF SELF LOOPS:", self_loop_count)

In [ ]:
print("NUMBER OF TRUST EDGES:", pos_count)

In [ ]:
print("NUMBER OF DISTRUST EDGES:", neg_count)

In [ ]:
print("NUMBER OF NODES IN TRIADS:",len(set([val for sublist in triads for val in sublist])))

In [ ]:
print("NUMBER OF NODES TOTAL:",len(G.nodes()))

In [ ]:
def get_trust_category(entry):
    categories = {
        (1,1,1) : "TTT",
        (-1,1,1) : "TTD",
        (-1,-1,1) : "TDD",
        (-1,-1,-1) : "DDD"
    }
    return categories[tuple(sorted([x[1] for x in entry]))]

for i in range(len(triads_and_weights)):
    triads_and_weights[i].append(get_trust_category(triads_and_weights[i]))

In [ ]:
col_format = tuple(zip(*triads_and_weights))
table = pd.DataFrame({
    "trust_category": col_format[3],
    "edge_1": tuple(zip(*col_format[0]))[0],
    "trust_1": tuple(zip(*col_format[0]))[1],
    "edge_2": tuple(zip(*col_format[1]))[0],
    "trust_2": tuple(zip(*col_format[1]))[1],
    "edge_3": tuple(zip(*col_format[2]))[0],
    "trust_3": tuple(zip(*col_format[2]))[1]
})

In [ ]:
table.sort_values(['trust_category'],ascending=False).reset_index(drop=True)